Get median ages

In [1]:
%reset -f

In [36]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import rpy2
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects.vectors import StrVector

from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [3]:
import os
cwd = os.getcwd()
data_dir = cwd.rsplit('/', 1)[0] +'/data'

In [4]:
data = robjects.r(f'read.csv("{data_dir}/PUMS_demographics_2019.csv", header=TRUE)')
data.head()

,person_id,PWGTP,RAC1P,HISP,NATIVITY,LANX,ENG,AGEP,PUMA,ST,...,PWGTP71,PWGTP72,PWGTP73,PWGTP74,PWGTP75,PWGTP76,PWGTP77,PWGTP78,PWGTP79,PWGTP80
1,20150000000461,20,Asian alone,Not Spanish/Hispanic/Latino,Foreign born,"Yes, speaks another language",Not well,45,4004,36,...,20,17,34,7,31,31,30,24,5,20
2,20150000000462,20,Asian alone,Not Spanish/Hispanic/Latino,Foreign born,"Yes, speaks another language",Not at all,49,4004,36,...,22,14,31,10,34,43,28,25,8,15
3,20150000000463,13,Asian alone,Not Spanish/Hispanic/Latino,Foreign born,"Yes, speaks another language",Very well,21,4004,36,...,12,13,23,4,25,23,15,12,3,11
4,20150000000464,18,Asian alone,Not Spanish/Hispanic/Latino,Native,"Yes, speaks another language",Very well,10,4004,36,...,23,18,31,6,30,34,18,15,4,16
5,20150000003431,14,White alone,Not Spanish/Hispanic/Latino,Native,"No, speaks only English",N/A (less than 5 years old/speaks only English),31,4012,36,...,21,3,4,12,14,12,12,24,4,20


In [5]:
survey_package = rpackages.importr('survey')
base = rpackages.importr('base')

In [6]:
rw_cols = [f"PWGTP{x}" for x in range(1, 81)]  # This will get refactored out
weight_col = "PWGTP"
geo_col = "PUMA"

In [7]:
survey_design = survey_package.svrepdesign(
    variables=data[["AGEP"]],
    repweights=data[rw_cols],
    weights=data[weight_col],
    combined_weights=True,
    type="other",
    scale=4 / 80,
    rscales=1,
)

In [8]:
survey_design

<rpy2.robjects.vectors.ListVector object at 0x7ffad317b0c0> [RTYPES.VECSXP]
R classes: ('svyrep.design',)
[StrSexpVe..., FloatSexp..., IntSexpVe..., NULLType, ..., IntSexpVe..., ListSexpV..., FloatSexp..., BoolSexpV...]
  type: <class 'rpy2.robjects.vectors.StrVector'>
  <rpy2.robjects.vectors.StrVector object at 0x7ffad52e3180> [RTYPES.STRSXP]
R classes: ('character',)
['other']
  scale: <class 'numpy.ndarray'>
  array([0.05])
  rscales: <class 'numpy.ndarray'>
  array([1], dtype=int32)
  rho: <class 'rpy2.rinterface_lib.sexp.NULLType'>
  <rpy2.rinterface_lib.sexp.NULLType object at 0x7ffb1d016080> [RTYPES.NILSXP]
...
  combined.weights: <class 'numpy.ndarray'>
  array([20, 20, 13, ..., 32, 31, 25], dtype=int32)
  variables: <class 'pandas.core.frame.DataFrame'>
          PWGTP1  PWGTP2  PWGTP3  PWGTP4  PWGTP5  PWGTP6  PWGTP7  PWGTP8  \
1           20      20       6      31       5      19      33      40   
2           16      21       6      33       6      19      38      38   
3           10      12       4      20       4      11      24      24   
4           16      26       5      28       6      17      28      29   
5           14      12      12       4      24      12      13       5   
...        ...     ...     ...     ...     ...     ...     ...     ...   
356540      19      29      18       5      17       5      42      26   
356541      46      25      26      28      50       8      40      31   
356542      53      28      30      30      54       8      46      36   
356543      48      27      32      29      47       9      47      33   
356544      35      21      24      23      37       7      35      26   

        PWGTP9  PWGTP10  ...  PWGTP71  PWGTP72  PWGTP73  PWGTP74  PWGTP75  \
1           31       22  ...       20       17       34        7       31   
2           28       19  ...       22       14       31       10       34   
3           20       12  ...       12       13       23        4       25   
4           24       20  ...       23       18       31        6       30   
5           11        3  ...       21        3        4       12       14   
...        ...      ...  ...      ...      ...      ...      ...      ...   
356540       6        6  ...       25       27       18       21       25   
356541       9        9  ...       48       30       10       45       32   
356542       9        9  ...       50       31       11       50       35   
356543       9        9  ...       49       29        9       44       31   
356544       7        7  ...       39       22        7       41       26   

        PWGTP76  PWGTP77  PWGTP78  PWGTP79  PWGTP80  
1            31       30       24        5       20  
2            43       28       25        8       15  
3            23       15       12        3       11  
4            34       18       15        4       16  
5            12       12       24        4       20  
...         ...      ...      ...      ...      ...  
356540       47       18        7       40       34  
356541       28       31       25       11       26  
356542       32       34       27       12       29  
356543       30       34       26       13       30  
356544       24       25       20        9       25  

[356544 rows x 80 columns]
  pweights: <class 'numpy.ndarray'>
  array([79.])
  repweights: <class 'rpy2.robjects.vectors.BoolVector'>
  <rpy2.robjects.vectors.BoolVector object at 0x7ffad5215340> [RTYPES.LGLSXP]
R classes: ('logical',)
[       0]

In [37]:
aggregated = survey_package.svyby(
        formula=data[["AGEP"]],
        by=data[[geo_col]],
        design=survey_design,
        quantiles=base.c(.5),
        FUN=survey_package.svyquantile,
     **{"interval.type": "quantile"},
    )

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all replicate weight designs give valid standard errors for quantiles.

R[write to console]: Not all r

In [40]:
aggregated.drop(columns = geo_col, inplace=True)

In [44]:
variable_col = 'AGEP'
aggregated.rename(columns={variable_col: f"{variable_col}-median", f'se.{variable_col}':f'{variable_col}-se'}, inplace=True)
aggregated


,AGEP-median,AGEP-se
3701,39.0,0.921615
3702,36.0,0.379967
3703,43.0,0.935080
3704,37.0,0.661438
3705,30.0,0.818153
3706,32.0,0.379967
3707,30.0,0.000000
3708,32.0,0.222205
3709,34.0,0.759934
3710,31.0,0.661438
